# Phase 1 - Setting everything up

Open this repository in a GitHub Codespace.
Before you start with anything else, make sure you setup the infrastructure required. Follow the readme file in the root folder to do this!

To start with Phase 1, if not already done run this in the top level folder:

```
pip install -r requirements.txt
```




In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

Found Azure OpenAI API Base Endpoint: https://cog-ukzrhgnqdd2wy.openai.azure.com/


To see if stuff works in general, you can run this.

In [6]:
response = client.chat.completions.create(
    model = model_name,    
    messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
)

print(response.choices[0].message.content)

the feeling of connection and understanding between people. There's something truly magical about that shared moment of empathy and harmony. How about you?


This is the object model for receiving questions.

In [7]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [12]:

async def ask_question(ask: Ask):
    # """
    # # Ask a question
    # """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    
    start_phrase =  ask.question
    response: openai.types.chat.chat_completion.ChatCompletion = None
    
    response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : "I'm a 5 year old and i have the following question. Please answer only in short answers:" + start_phrase}, 
                     { "role" : "system", "content" : "Answer this question with content only."}]
    )

    print(response.choices[0].message.content)
    print(response)
    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [13]:
import logging

errors = 0

def check_answer(answer, expected):
    if answer != expected:
        logging.error(f"Got answer '{answer}', but expected '{expected}'")
    return 1
ask = Ask(question="Which movie features a plot where a young girl named Dorothy is transported to a magical land via a tornado? 1) Cinderella 2) The Wizard of Oz 3) Alice in Wonderland 4) The Little Mermaid", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "The Wizard of Oz")

ask = Ask(question="Is Yoda a character from the Star Trek universe: True or False?", type=QuestionType.true_or_false)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "false")

ask = Ask(question="How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?", type=QuestionType.estimation)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "3")

ask = Ask(question="How many states does the US have", type=QuestionType.estimation)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "50")

ask = Ask(question="Is an apple a fruit?", type=QuestionType.true_or_false)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "true")

ask = Ask(question="Who was the first president of the United States", type=QuestionType.estimation)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "George Washington")

ask = Ask(question="Which animal is extinct? 1) Dog 2) T. Rex 3) Cat", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
errors+=check_answer(answer.answer, "T. Rex")


if errors > 0:
    raise Exception(f"Got {errors} errors")

Sending a request to openai
The Wizard of Oz
ChatCompletion(id='chatcmpl-A8OE7aXCkRE5e2LTbARt7PHMKftK4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Wizard of Oz', refusal=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726563115, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=4, prompt_tokens=82, total_tokens=86), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {}}])
Answer: answer='The Wizard of Oz' correlationToken=None

# Sample Questions
Sample Questions could look like this. Make sure your answer exactly  matches the required answer.



{
    "id": 3,
    "phase": 1,
    "question": "Which movie features a plot where a young girl named Dorothy is transported to a magical land via a tornado? 1) Cinderella 2) The Wizard of Oz 3) Alice in Wonderland 4) The Little Mermaid",
    "answer": "The Wizard of Oz",
    "type": "multiple_choice"
},
{
    "id": 4,
    "phase": 1,
    "question": "Is Yoda a character from the Star Trek universe: True or False?",
    "answer": false,
    "type": "true_or_false"
},
{
    "id": 5,
    "phase": 1,
    "question": "How many movies are there in 'The Lord of the Rings' trilogy directed by Peter Jackson?",
    "answer": 3,
    "type": "estimation"
}

Make sure you transfer your code changes into main.py (or additional files). 
You can test your app locally using uvicorn. (See Readme.md for details.)

Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase1 $AZURE_ENV_NAME
```
